In [1]:
from datasets import load_dataset

dataset = load_dataset("ccdv/arxiv-summarization")
dataset

c:\Anaconda\envs\sf2023\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No config specified, defaulting to: arxiv-summarization/section
Found cached dataset arxiv-summarization (C:/Users/Justin Du/.cache/huggingface/datasets/ccdv___arxiv-summarization/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3)
100%|██████████| 3/3 [00:00<00:00,  5.35it/s]


DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 203037
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6436
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6440
    })
})

In [2]:
data_train = dataset['train']
data_test = dataset['test']
data_val = dataset['validation']
data_test

Dataset({
    features: ['article', 'abstract'],
    num_rows: 6440
})

In [3]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq

checkpoint = 'facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [4]:
max_length = 1024

def tokenize_function(data):

    model_inputs = tokenizer(
        data["article"],
        truncation=True,
        max_length=max_length,
    )


    labels = tokenizer(
        data['abstract'],
        truncation=True,
        max_length=max_length
    )

    model_inputs["decoder_input_ids"] = labels["input_ids"]
    model_inputs["decoder_attention_mask"] = labels["attention_mask"]
    return model_inputs

In [5]:
tok = data_test.map(tokenize_function, batched=True)
tok

Loading cached processed dataset at C:/Users/Justin Du/.cache/huggingface/datasets/ccdv___arxiv-summarization/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3\cache-aa89378e5dcd8e81.arrow


Dataset({
    features: ['article', 'abstract', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
    num_rows: 6440
})

In [6]:
testx = tok['input_ids'][0:3]

In [7]:
print(testx[0])

[0, 1990, 59, 291, 107, 5, 936, 9, 3611, 9, 765, 111, 1385, 1022, 9, 4118, 1940, 34, 57, 1687, 18808, 479, 1437, 50118, 171, 3725, 8069, 5, 765, 111, 1385, 27185, 2192, 9, 5, 1337, 21383, 9, 4118, 1940, 479, 1437, 50118, 484, 27185, 2192, 58, 12333, 2156, 53, 5, 27185, 2192, 59, 18918, 360, 8, 31, 5, 22455, 9, 787, 1178, 40051, 246, 68, 27779, 360, 36, 787, 1178, 40051, 306, 68, 27779, 107, 4839, 32, 2801, 144, 747, 479, 1437, 50118, 78, 9, 106, 21, 2967, 30, 787, 45421, 1459, 11, 5, 37627, 2407, 15285, 731, 9, 42685, 111, 33803, 34968, 12333, 30, 5, 42685, 111, 33803, 19416, 5638, 5906, 10009, 5, 18134, 4118, 4532, 2511, 36, 5278, 119, 4839, 479, 1437, 50118, 42, 675, 24414, 21, 1474, 13, 97, 4118, 34968, 414, 8, 13, 5, 276, 86, 675, 787, 45421, 1459, 479, 1437, 50118, 24, 21, 67, 303, 11, 1759, 1054, 34968, 148, 4118, 16726, 753, 8, 291, 787, 45421, 1459, 2156, 53, 24, 21, 45, 303, 11, 5, 4118, 34968, 414, 148, 4118, 16726, 820, 787, 45421, 1459, 479, 1437, 50118, 18134, 1437, 1437, 

In [8]:
import torch

In [9]:
asdf = []
for i in range(len(testx)):
    asdf.append(list(testx[i]))
asdf = torch.as_tensor(asdf)

In [10]:
print(asdf)

tensor([[   0, 1990,   59,  ...,   32, 4756,    2],
        [   0,  405,   16,  ...,  143,  787,    2],
        [   0,  281,   10,  ..., 1437, 1437,    2]])


In [11]:
outputs = model.generate(asdf, max_length=150, length_penalty=2.0, num_beams=4, early_stopping=True)

In [12]:
print(tokenizer.decode(outputs[0]))

</s><s>The problem of properties of short - term changes of solar activity has been considered extensively. Several periodicities were detected, but the periodicities about 155 days and from the interval of @xmath3 $ ] days are mentioned most often. The power of this periodicity started growing at cycle 19, decreased in cycles 20 and 21 and disappered after cycle 21.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [13]:
print(tokenizer.decode(outputs[1]))

</s><s>It is believed that the direct detection of gravitational waves ( gws) will bring the era of gravitational wave astronomy. Interferometer detectors are now under operation and awaiting the first signal of gws. pulsar timing arrays ( ptas) can be used to search for very low frequency gravitational waves. The promising sources are super massive black hole binaries and cosmic ( super)string.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [14]:
print(tokenizer.decode(outputs[2]))

</s><s>The tunneling through a potential barrier plays a very important role in the microscopic world and has been studied extensively since the birth of quantum mechanics. For most of the potential barriers, the penetrability can not be calculated analytically. In the present work, we derived a new barrier penetration formula based on the wkb approximation. We apply this new formula to evaluate @xmath0 decay half - lives of atomic nuclei.</s>


In [ ]:
import evaluate
rouge_score = evaluate.load('rouge')

In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8
logging_steps = len(data_val)
training_args = Seq2SeqTrainingArguments(
    output_dir='test-trainer',
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps
)

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tok['test'],
    eval_dataset=tok['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
# samples = tok['test'][:2000]
# samples = {k: v for k, v in samples.items() if k not in ['abstract', 'article']}
# batch = data_collator(samples)

In [ ]:
# chunk = 500
# tok_abs = []

# for i in (data_test[pos:pos + chunk] for pos in range(0, len(data_test), chunk)):
#     tok_abs.append(tokenizer(i['abstract'], truncation=True, padding=True, return_tensors='tf'))


In [ ]:
# tok_art = []
# for i in (data_test[pos:pos + chunk] for pos in range(0, len(data_test), chunk)):
#     tok_art.append(tokenizer(i['article'], truncation=True, ))

In [ ]:
# import torch
# tok = torch.cat(tok_abs, dim=1)


In [ ]:
# test_tok_abs = tokenizer(data_train['article'], truncation=True, padding='max_length')